In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [6]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Primer Cuatrimestre de 2018. Examen parcial, tercera oportunidad

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un archivo con el siguiente formato: (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).

Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las fechas se encuentran en el formato YYYY-MM-DD. 

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a)  Las categorías que hayan incrementado su tasa de resolución en al menos un 10%  en el último trimestre, con respecto al trimestre anterior.
b) Tasa de participación de Batman por categoría, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto, y robo de vehículos) 

In [7]:
# (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).
cases = [("2019-01-01", 1, "case 1", 2, "otro delito", -1, -1), 
         ("2019-06-01", 2, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 3, "case 2", 3, "robo", -1, -1),         
         ("2019-06-01", 4, "case 2", 1, "robo", -1, -1),         
         ("2019-06-01", 5, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 6, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 9, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 10, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 60, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 70, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 80, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 90, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 100, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 600, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 700, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 800, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 900, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 1000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 6000, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7000, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8000, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 9000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 10000, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 92, "case 2", 2, "hurto", -1, -1),
         ("2019-06-01", 93, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 94, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 95, "case 2", 3, "hurto", -1, -1),
         ("2019-09-01", 96, "case 2", 2, "hurto", -1, -1),
 
        ]

# (id_caso, respuesta)
batsignal = [(1,1),
         (2,1),
         (3,0),
         (4,0),
         (5,1),
         (6,0),
         (7,1),
         (8,0),
         (9,1),
         (100,1),
         (600,0),
         (700,1),
         (800,0),
         (900,1),
         (1000,1),
         (6000,0),
         (7000,1),
         (8000,0),
         (9000,1),
         (10000,1),
         (92,0),
         (93,0),             
         (94,0),
         (95,0),             
         (96,1)             
        ]

In [8]:
casesRdd = sc.parallelize(cases)
batsignalRdd = sc.parallelize(batsignal)

a)  Las categorías que hayan incrementado su tasa de resolución en al menos un 10%  en el último trimestre, con respecto al trimestre anterior.

In [59]:
def en_periodo(x):
    # Consideramos (por los datos) los trimestres 2 (abril a junio)y 3(julio a septiembre)
    return pd.to_datetime("2019-04-01")<=pd.to_datetime(x[0])<pd.to_datetime("2019-10-01")

def clasificar_categoria_periodo_resueltos(x):
    """ 
    Input: [fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud]
    Devuelve: ((categoria,periodo),(es_caso_resuelto,1))"""
    categoria = x[4]
    fecha = pd.to_datetime(x[0])
    return ((categoria,0 if fecha<pd.to_datetime("2019-07-01") else 1),(1 if x[3] == 2 else 0,1))

In [60]:
#filtramos los que no son de esos trimestres
casos_periodo = casesRdd.filter(lambda x: x[3] in [2,3])\
                        .filter(en_periodo)\
                        .map(clasificar_categoria_periodo_resueltos)
casos_periodo.collect()

[(('robo', 0), (1, 1)),
 (('robo', 0), (0, 1)),
 (('robo', 0), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (1, 1)),
 (('hurto', 0), (1, 1)),
 (('hurto', 0), (0, 1)),
 (('hurto', 0), (0, 1)),
 (('hurto', 0), (0, 1)),
 (('hurto', 1), (1, 1))]

In [61]:
casos_periodo_total = casos_periodo.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
# ((categoria,periodo),(total_resueltos,total_casos))
casos_periodo_total.collect()

[(('robo', 0), (2, 3)),
 (('hurto', 1), (1, 1)),
 (('robo', 1), (10, 15)),
 (('hurto', 0), (1, 4))]

In [55]:
casos_por_categoria = casos_periodo_total.map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey().map(lambda x: (x[0],dict(x[1])))

casos_por_categoria.collect()

[('hurto', {1: (1, 1), 0: (1, 4)}), ('robo', {0: (2, 3), 1: (10, 15)})]

In [84]:
def tasa_aumento_diez_porciento(x):
    """
    Input : {periodo:(casos_resueltos,casos_totales)}
    Periodos: 0 o 1
    """
    return (x[1][0]/x[1][1]-x[0][0]/x[0][1])>0.1

In [85]:
casos_por_categoria.filter(lambda x: tasa_aumento_diez_porciento(x[1])).map(lambda x: x[0]).collect()

['hurto']

b) Tasa de participación de Batman por categoría, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto, y robo de vehículos) 

In [90]:
casos_batman_asiste = batsignalRdd.filter(lambda x: x[1] == 1)
#ya esta en formato (id_caso,asiste)
casos_batman_asiste.collect()

[(1, 1),
 (2, 1),
 (5, 1),
 (7, 1),
 (9, 1),
 (100, 1),
 (700, 1),
 (900, 1),
 (1000, 1),
 (7000, 1),
 (9000, 1),
 (10000, 1),
 (96, 1)]

In [97]:
delitos_contra_la_propiedad = ['incendio intencional','robo','hurto','robo de vehículos']
casos_delitos_contra_la_propiedad = casesRdd.filter(lambda x: x[4] in delitos_contra_la_propiedad).map(lambda x: (x[1],x[4]))
casos_asistencia = casos_delitos_contra_la_propiedad.leftOuterJoin(casos_batman_asiste)
casos_asistencia.collect()


[(8, ('robo', None)),
 (80, ('robo', None)),
 (600, ('robo', None)),
 (800, ('robo', None)),
 (1000, ('robo', 1)),
 (6000, ('robo', None)),
 (7000, ('robo', 1)),
 (8000, ('robo', None)),
 (9000, ('robo', 1)),
 (10000, ('robo', 1)),
 (96, ('hurto', 1)),
 (9, ('robo', 1)),
 (2, ('robo', 1)),
 (10, ('robo', None)),
 (90, ('robo', None)),
 (3, ('robo', None)),
 (4, ('robo', None)),
 (60, ('robo', None)),
 (100, ('robo', 1)),
 (700, ('robo', 1)),
 (900, ('robo', 1)),
 (92, ('hurto', None)),
 (5, ('robo', 1)),
 (93, ('hurto', None)),
 (6, ('robo', None)),
 (70, ('robo', None)),
 (94, ('hurto', None)),
 (7, ('robo', 1)),
 (95, ('hurto', None))]

In [103]:
casos_asistencia\
    .map(lambda x : (x[1][0],(1 if x[1][1] == 1 else 0,1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0],x[1][0]/x[1][1])).collect()

[('robo', 0.4583333333333333), ('hurto', 0.2)]